In [2]:
import pandas as pd
import datetime

In [3]:
date_str = datetime.datetime.now().strftime("%Y-%m-%d")

date_str = '2018-05-20'

print("Date: " + date_str)

Date: 2018-05-20


In [5]:
# Read the Calendar extract to get additional details
cal_1 = pd.read_json('/Users/taj/GitHub/scraping/stayz/WebData/nsw_calendar/stayz_calendar_' + date_str + '.json')



cal_2 = cal_1[cal_1['property_id'].notnull()]

cal_2.set_index('property_id',inplace=True)


cal = cal_2

cal.head()

,calendar,ext_at,features,photos,review_count,review_value
property_id,,,,,,
9191650,"<thead><tr><th scope=""col""><span title=""Monday...",2018-05-20 09:19:10,Air conditioning\nBalcony\nBooks\nChild's high...,22,25,5.0
9004173,"<thead><tr><th scope=""col""><span title=""Monday...",2018-05-20 09:19:12,Books\nChild's high chair\nChildren welcome\nC...,17,54,4.9
9067504,"<thead><tr><th scope=""col""><span title=""Monday...",2018-05-20 09:19:15,Air conditioning\nBooks\nChild's high chair\nC...,23,38,5.0
9197130,"<thead><tr><th scope=""col""><span title=""Monday...",2018-05-20 09:19:17,Air conditioning\nBalcony\nBooks\nChild's high...,20,25,5.0
9188751,"<thead><tr><th scope=""col""><span title=""Monday...",2018-05-20 09:19:19,Balcony\nBooks\nChild's high chair\nChildren w...,20,23,5.0


In [6]:
position_dict = {}
position_index = 0


data_dict = {}


# Step one - Create the dictionary of all possible feature items
for index, row in cal.iterrows():
    
    a = row['features']
    
    #print("Features: " + a)
    #save_features = [0] * len(position_dict)
    
    b = a.split('\n')
    
    for b1 in b:
        # Check if it exists in the dictionary
        
        if ( len(b1) > 0 ):
            if b1 in position_dict:
                b1_pos = position_dict[b1]
                #save_features[b1_pos] = 1

            else:
                position_dict[b1] = position_index

                b1_pos = position_index

                position_index += 1
            
print(position_dict)



{'Air conditioning': 0, 'Balcony': 1, 'Books': 2, "Child's high chair": 3, 'Children welcome': 4, 'Coffee maker': 5, 'Dining': 6, 'Dishes and utensils': 7, 'Dishwasher': 8, 'Games': 9, 'Garage': 10, 'Internet access': 11, 'Iron and board': 12, 'Kitchen': 13, 'Lawn/Garden': 14, 'Linens provided': 15, 'Living room': 16, 'Microwave': 17, 'Non-Smoking': 18, 'Outdoor grill': 19, 'Oven': 20, 'Parking': 21, 'Private pool': 22, 'Refrigerator': 23, 'Satellite/Cable': 24, 'Stove': 25, 'Toaster': 26, 'Towels provided': 27, 'Washing machine': 28, 'Clothes dryer': 29, 'Deck/Patio': 30, 'DVD player': 31, 'Hair dryer': 32, 'Heating': 33, 'Hot tub': 34, 'Pets allowed': 35, 'Telephone': 36, 'Television': 37, 'Toys': 38, 'Wood stove': 39, 'Stereo': 40, 'Tennis': 41, 'Video library': 42, 'Fireplace': 43, 'Wheelchair accessible': 44, 'Games room': 45, 'Video Games': 46, 'Long-term renters welcome': 47, 'Fitness room/Equipment': 48, 'Heated pool': 49, 'Events allowed': 50, 'Communal pool': 51, 'Ceiling fan

In [7]:

# Step 2 - scan and create the data
for index, row in cal.iterrows():
    #print(index)
    a = row['features']
    b = a.split('\n')
    
    save_features = [0] * len(position_dict)
    
    for b1 in b:    
        if ( len(b1) > 0 ):
            if b1 in position_dict:
                b1_pos = position_dict[b1]
                save_features[b1_pos] = 1
            else:
                print("ERROR: found a new feature not in dictionary: '" + b1 + "'")
    #print("Index: " + index)
    #if ( index > 0 ):
    data_dict[index] = save_features
    
df1 = pd.DataFrame.from_dict(data_dict)


df2 = df1.T

col_n_1 = position_dict.keys()

col_n_2 = []


for c in col_n_1:
    # Cleaning text
    for char in '\'/-.,\n ':
        c=c.replace(char,'_')

    # All words lower case
    c = c.lower()
    
    col_n_2.append('f_' + c)

df2.columns = col_n_2

df2.head()

,f_air_conditioning,f_balcony,f_books,f_child_s_high_chair,f_children_welcome,f_coffee_maker,f_dining,f_dishes_and_utensils,f_dishwasher,f_games,...,f_events_allowed,f_communal_pool,f_ceiling_fan,f_indoor_pool,f_sauna,f_pantry_items,f_cot,f_eco_friendly,f_free_wifi,f_free_fixed_line_internet
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100012,0,0,0,0,1,0,1,1,1,0,...,0,0,1,0,0,0,0,0,0,0
100171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100172,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100173,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Write out the file to CSV so it can be joined in the main branch

csv_out = df2.to_csv()

fh = open('/Users/taj/GitHub/scraping/stayz/WebData/nsw_features/stayz_nsw_features_' + date_str + '.csv', 'w')

fh.write(csv_out)

fh.close()